In [2]:
import numpy as np
import json

In [5]:
train_path = './data/train_data.json'
dev_path = '../data/dev_data.json'
schema_path = '../data/all_50_schemas'

In [6]:
train = []
with open(train_path, 'r', encoding='utf8') as f:
    for line in f:
        train.append(json.loads(line.strip()))

FileNotFoundError: [Errno 2] No such file or directory: './data/train_data.json'

In [ ]:
import pprint
pp = pprint.PrettyPrinter(inden

In [14]:
pp.pprint(train[np.random.randint(len(train))])

{   'postag': [   {'pos': 't', 'word': '现在'},
                  {'pos': 'w', 'word': '，'},
                  {'pos': 'v', 'word': '历经'},
                  {'pos': 'n', 'word': '鑫涛'},
                  {'pos': 'w', 'word': '（'},
                  {'pos': 'nr', 'word': '平鑫涛'},
                  {'pos': 'w', 'word': '，'},
                  {'pos': 'nr', 'word': '琼瑶'},
                  {'pos': 'u', 'word': '的'},
                  {'pos': 'n', 'word': '丈夫'},
                  {'pos': 'w', 'word': '）'},
                  {'pos': 'v', 'word': '失智'},
                  {'pos': 'w', 'word': '、'},
                  {'pos': 'nz', 'word': '中风'},
                  {'pos': 'w', 'word': '、'},
                  {'pos': 'vn', 'word': '插管'},
                  {'pos': 'u', 'word': '的'},
                  {'pos': 'n', 'word': '风波'},
                  {'pos': 'f', 'word': '后'},
                  {'pos': 'w', 'word': '，'},
                  {'pos': 'r', 'word': '我'},
                  {'pos': 'v', 'word': '

In [15]:
dev = []
with open(dev_path, 'r', encoding='utf8') as f:
    for line in f:
        dev.append(json.loads(line.strip()))

In [16]:
pp.pprint(dev[np.random.randint(len(dev))])

{   'postag': [   {'pos': 'nr', 'word': '张际林'},
                  {'pos': 'w', 'word': '，'},
                  {'pos': 'a', 'word': '男'},
                  {'pos': 'w', 'word': '，'},
                  {'pos': 'nz', 'word': '汉族'},
                  {'pos': 'w', 'word': '，'},
                  {'pos': 't', 'word': '1930年'},
                  {'pos': 't', 'word': '1月'},
                  {'pos': 'v', 'word': '出生'},
                  {'pos': 'p', 'word': '于'},
                  {'pos': 'ns', 'word': '河北省'},
                  {'pos': 'ns', 'word': '迁安市'}],
    'spo_list': [   {   'object': '汉族',
                        'object_type': 'Text',
                        'predicate': '民族',
                        'subject': '张际林',
                        'subject_type': '人物'},
                    {   'object': '河北省迁安市',
                        'object_type': '地点',
                        'predicate': '出生地',
                        'subject': '张际林',
                        'subject_type': '人物'},
 

In [17]:
schemas = []
with open(schema_path, 'r', encoding='utf8') as f:
    for line in f:
        schemas.append(json.loads(line.strip()))

In [18]:
schemas

[{'object_type': '地点', 'predicate': '祖籍', 'subject_type': '人物'},
 {'object_type': '人物', 'predicate': '父亲', 'subject_type': '人物'},
 {'object_type': '地点', 'predicate': '总部地点', 'subject_type': '企业'},
 {'object_type': '地点', 'predicate': '出生地', 'subject_type': '人物'},
 {'object_type': '目', 'predicate': '目', 'subject_type': '生物'},
 {'object_type': 'Number', 'predicate': '面积', 'subject_type': '行政区'},
 {'object_type': 'Text', 'predicate': '简称', 'subject_type': '机构'},
 {'object_type': 'Date', 'predicate': '上映时间', 'subject_type': '影视作品'},
 {'object_type': '人物', 'predicate': '妻子', 'subject_type': '人物'},
 {'object_type': '音乐专辑', 'predicate': '所属专辑', 'subject_type': '歌曲'},
 {'object_type': 'Number', 'predicate': '注册资本', 'subject_type': '企业'},
 {'object_type': '城市', 'predicate': '首都', 'subject_type': '国家'},
 {'object_type': '人物', 'predicate': '导演', 'subject_type': '影视作品'},
 {'object_type': 'Text', 'predicate': '字', 'subject_type': '历史人物'},
 {'object_type': 'Number', 'predicate': '身高', 'subject_type':